In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
'''
### code to concat all the scraped data ###

data = pd.DataFrame()
list_country = ['indonesia', 'malaysia', 'singapore', 'australia', 'new zealand']
list_job = ['da', 'ds']
list_category = ['kategori1', 'kategori2', 'kategori3', 'kategori4', 'kategori5']
for country in list_country:
  for  job in list_job:
    if country == 'new zealand':
      path = f"/content/gdrive/MyDrive/WEB SCRAPING 101/{country}/{job}_newzealand.csv"
      new_data = pd.read_csv(path)
      data = pd.concat([data, new_data])
    else:
      for category in list_category:
        path = f"/content/gdrive/MyDrive/WEB SCRAPING 101/{country}/{job}_{category}_{country}.csv"
        new_data = pd.read_csv(path)
        data = pd.concat([data, new_data])

data.to_csv('/content/gdrive/My Drive/data_full.csv', index=False)
'''

'\n### code to concat all the scraped data ###\n\ndata = pd.DataFrame()\nlist_country = [\'indonesia\', \'malaysia\', \'singapore\', \'australia\', \'new zealand\']\nlist_job = [\'da\', \'ds\']\nlist_category = [\'kategori1\', \'kategori2\', \'kategori3\', \'kategori4\', \'kategori5\']\nfor country in list_country:\n  for  job in list_job:\n    if country == \'new zealand\':\n      path = f"/content/gdrive/MyDrive/WEB SCRAPING 101/{country}/{job}_newzealand.csv"\n      new_data = pd.read_csv(path)\n      data = pd.concat([data, new_data])\n    else:\n      for category in list_category:\n        path = f"/content/gdrive/MyDrive/WEB SCRAPING 101/{country}/{job}_{category}_{country}.csv"\n        new_data = pd.read_csv(path)\n        data = pd.concat([data, new_data])\n\ndata.to_csv(\'/content/gdrive/My Drive/data_full.csv\', index=False)\n'

## 1. Data Preparation

In [4]:
data = pd.read_csv('/content/gdrive/MyDrive/WEB SCRAPING 101/data_web_scraping_full_new.csv')

In [ ]:
data.head(2)

,job_title,company_name,location,date,seniority,emp_type,job_func,industries,job_desc,job_link,follower,company_size,applicant_total,work_from,date_scraped
0,Data Science Trainee,Algoritma Data Science School,"Jakarta, Jakarta, Indonesia",2023-07-01,Internship,Full-time,Engineering and Information Technology,Education Administration Programs,Background\n\n\n\n\nAre you looking to build a...,https://id.linkedin.com/jobs/view/data-science...,"11,560 followers",11-50 employees · Education Administration Pro...,155 applicants,"Algoritma Data Science School · Jakarta, Jakar...",2023-8-3
1,Data Intelligence Intern,SIRCLO,"Tangerang, Banten, Indonesia",2023-08-02,Internship,Full-time,Information Technology,IT Services and IT Consulting,nullResponsibilities\n\n\n\n\nSIRCLO Internshi...,https://id.linkedin.com/jobs/view/data-intelli...,"67,415 followers","1,001-5,000 employees · IT Services and IT Con...",126 applicants,"SIRCLO · Tangerang, Banten, Indonesia (Hybrid)...",2023-8-3


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18310 entries, 0 to 18309
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        18310 non-null  object
 1   company_name     18310 non-null  object
 2   location         18310 non-null  object
 3   date             18310 non-null  object
 4   seniority        18310 non-null  object
 5   emp_type         17805 non-null  object
 6   job_func         17805 non-null  object
 7   industries       16813 non-null  object
 8   job_desc         18310 non-null  object
 9   job_link         18310 non-null  object
 10  follower         16727 non-null  object
 11  company_size     18284 non-null  object
 12  applicant_total  18086 non-null  object
 13  work_from        18285 non-null  object
 14  date_scraped     18310 non-null  object
dtypes: object(15)
memory usage: 2.1+ MB


In [ ]:
data.duplicated().sum()

0

In [ ]:
# drop duplicates and drop some columns
data.drop_duplicates(inplace=True)
#data.drop(['job_func'], axis=1, inplace=True)

# rename columns dan rearrange columns
data.rename(columns={
    "date": "date_posted", "follower": "company_followers",
    'applicant_total': 'total_applicants', 'work_from': 'work_place'
    },
    inplace = True)
data = data[['job_title', 'company_name', 'location', 'date_posted', 'date_scraped', 'seniority',
             'emp_type', 'industries', 'company_followers', 'company_size',
             'total_applicants', 'work_place', 'job_link', 'job_desc']]

In [ ]:
data.isna().sum()

job_title               0
company_name            0
location                0
date_posted             0
date_scraped            0
seniority               0
emp_type              505
industries           1497
company_followers    1583
company_size           26
total_applicants      224
work_place             25
job_link                0
job_desc                0
dtype: int64

In [ ]:
data['emp_type'].fillna('Unidentified', inplace = True)
data['industries'].fillna('Unidentified', inplace = True)
data['work_place'].fillna('Unidentified', inplace = True)
data['company_followers'].fillna('0 followers', inplace = True)
data['company_size'].fillna('No data', inplace = True)
data['total_applicants'].fillna('0 applicants', inplace = True)
data['total_applicants'] = data['total_applicants'].replace('', '0 applicants')
data.loc[~data['total_applicants'].str.contains('applicant'), 'total_applicants'] = '0 applicants'

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18310 entries, 0 to 18309
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   job_title          18310 non-null  object
 1   company_name       18310 non-null  object
 2   location           18310 non-null  object
 3   date_posted        18310 non-null  object
 4   date_scraped       18310 non-null  object
 5   seniority          18310 non-null  object
 6   emp_type           18310 non-null  object
 7   industries         18310 non-null  object
 8   company_followers  18310 non-null  object
 9   company_size       18310 non-null  object
 10  total_applicants   18310 non-null  object
 11  work_place         18310 non-null  object
 12  job_link           18310 non-null  object
 13  job_desc           18310 non-null  object
dtypes: object(14)
memory usage: 2.1+ MB


## 1.1 Cleaning `company_followers` and `total_applicants` columns

In [ ]:
# function to get rid of alphabet for company_followers and total_applicants

def remove_str(n):
  value = re.sub('\D', '', n)
  return value

In [ ]:
data[~data['total_applicants'].str.contains('applicant')]

,job_title,company_name,location,date_posted,date_scraped,seniority,emp_type,industries,company_followers,company_size,total_applicants,work_place,job_link,job_desc


In [ ]:
data['total_applicants'].value_counts()

0 applicants      1260
1 applicant        767
2 applicants       622
3 applicants       604
4 applicants       504
                  ... 
555 applicants       1
885 applicants       1
342 applicants       1
577 applicants       1
330 applicants       1
Name: total_applicants, Length: 770, dtype: int64

In [ ]:
data['company_followers'] = data['company_followers'].apply(remove_str)
data['company_followers'] = data['company_followers'].astype('int')

In [ ]:
data['total_applicants'] = data['total_applicants'].apply(remove_str)
data['total_applicants'] = data['total_applicants'].astype('int')

## 1.2 Cleaning `company_size` columns

In [ ]:
data['company_size'].value_counts()

1,001-5,000 employees · IT Services and IT Consulting                                   1464
10,001+ employees · IT Services and IT Consulting                                       1164
11-50 employees · Staffing and Recruiting                                                800
51-200 employees · IT Services and IT Consulting                                         530
51-200 employees · Staffing and Recruiting                                               448
                                                                                        ... 
501-1,000 employees · Think Tanks                                                          1
See recent hiring trends for ALIPAY LABS (SINGAPORE) PTE. LTD.. Try Premium for free       1
Wireless Services                                                                          1
See recent hiring trends for PARADIGM RECRUITMENT PTE. LTD.. Try Premium for free          1
11-50 employees · Civil Engineering                                   

In [ ]:
def clean_size(x):
  if str(x)[0].isupper():
    return 'No Data'
  else:
    value = re.sub('[A-Za-z. ·&/ ,]', '', x)
    value = value+' Employee'
    value = re.sub(r"-\s+", ' ', value)
    return value

In [ ]:
data['company_size'] = data['company_size'].apply(clean_size)

In [ ]:
data.company_size.value_counts()

10001+ Employee        3970
1001-5000 Employee     3190
11-50 Employee         2469
51-200 Employee        2308
No Data                1586
201-500 Employee       1458
5001-10000 Employee    1417
501-1000 Employee      1039
1-10 Employee           873
Name: company_size, dtype: int64

## 1.3 Cleaning `work_place` column

In [ ]:
data.sample(5)

,job_title,company_name,location,date_posted,date_scraped,seniority,emp_type,industries,company_followers,company_size,total_applicants,work_place,job_link,job_desc
1322,Senior DevOps Engineer,Multiplier,Indonesia,2023-06-16,2023-8-4,Mid-Senior level,Full-time,Software Development,138785,201-500 Employee,89,Multiplier · Indonesia (Remote) 1 month ago ...,https://id.linkedin.com/jobs/view/senior-devop...,Multiplier is APAC's 1st global talent enablem...
13342,Data Analyst,Hiway Group (Hiway Stabilizers) The Recycling ...,"Bayswater, Victoria, Australia",2023-07-28,2023-8-7,Associate,Full-time,Construction,2142,51-200 Employee,136,Hiway Group (Hiway Stabilizers) The Recycling ...,https://au.linkedin.com/jobs/view/data-analyst...,"Hiway Group, are recycling and reuse specialis..."
12757,Linux Engineer,Launch Recruitment,"Sydney, New South Wales, Australia",2023-08-02,2023-8-8,Entry level,Contract,"IT Services and IT Consulting, Software Develo...",14243,51-200 Employee,9,"Launch Recruitment · Sydney, New South Wales, ...",https://au.linkedin.com/jobs/view/linux-engine...,About The Role\n\n\n\n\nYou will be an experie...
2404,Flutter Developer,Turing,Indonesia,2023-04-22,2023-8-2,Entry level,Full-time,Software Development,277057,1001-5000 Employee,26,Turing · Indonesia (Remote) 3 months ago · 2...,https://id.linkedin.com/jobs/view/flutter-deve...,A well-established company on a mission to res...
16395,Data Center Engineer,NTT Ltd.,"Sydney, New South Wales, Australia",2023-07-09,2023-8-7,Mid-Senior level,Full-time,IT Services and IT Consulting,470047,10001+ Employee,66,"NTT Ltd. · Sydney, New South Wales, Australia ...",https://au.linkedin.com/jobs/view/data-center-...,Work across Azure workload migration initiativ...


In [ ]:
def clean_workplace(row):

  no_punc = re.sub(r'[^\w\s]', '', row)
  if len(re.findall(r'Hybrid', row)) != 0:
    return 'Hybrid'
  elif len(re.findall(r'Remote', row)) != 0:
    return 'Remote'
  elif len(re.findall(r'On-site', row)) != 0:
    return 'Onsite'
  else:
    return 'None'

In [ ]:
data['work_place'] = data['work_place'].apply(clean_workplace)

In [ ]:
data.work_place.value_counts()

Onsite    9174
Hybrid    4846
Remote    2574
None      1716
Name: work_place, dtype: int64

## 1.4 Cleaning `location` column

In [ ]:
data.head()

,job_title,company_name,location,date_posted,date_scraped,seniority,emp_type,industries,company_followers,company_size,total_applicants,work_place,job_link,job_desc
0,Data Science Trainee,Algoritma Data Science School,"Jakarta, Jakarta, Indonesia",2023-07-01,2023-8-3,Internship,Full-time,Education Administration Programs,11560,11-50 Employee,155,Onsite,https://id.linkedin.com/jobs/view/data-science...,Background\n\n\n\n\nAre you looking to build a...
1,Data Intelligence Intern,SIRCLO,"Tangerang, Banten, Indonesia",2023-08-02,2023-8-3,Internship,Full-time,IT Services and IT Consulting,67415,1001-5000 Employee,126,Hybrid,https://id.linkedin.com/jobs/view/data-intelli...,nullResponsibilities\n\n\n\n\nSIRCLO Internshi...
2,Business Analyst (intern),Kawan Lama Group,"West Jakarta, Jakarta, Indonesia",2023-07-24,2023-8-3,Internship,Internship,Retail,150260,10001+ Employee,292,Onsite,https://id.linkedin.com/jobs/view/business-ana...,Gambaran Pekerjaan\n\n\n\n\nMelakukan analisa ...
3,Business Intelligence Analyst Intern,Flash Coffee,"Jakarta, Jakarta, Indonesia",2023-06-27,2023-8-3,Internship,Internship,"Technology, Information and Internet",30563,501-1000 Employee,228,Onsite,https://id.linkedin.com/jobs/view/business-int...,Flash Coffee is one of Asia's fastest-growing ...
4,Data Scientist Intern,Amartha,"Jakarta, Jakarta, Indonesia",2023-07-27,2023-8-3,Internship,Full-time,Financial Services,64041,5001-10000 Employee,184,Hybrid,https://id.linkedin.com/jobs/view/data-scienti...,nullResponsibilities\n\n\n\n\nComplete ad hoc ...


In [ ]:
'''
countries = pd.DataFrame()
country_list = ['Australia', 'Indonesia', 'Malaysia', 'New Zealand', 'Singapore']

for country in country_list:
  path = f"/content/gdrive/MyDrive/WEB SCRAPING 101/provinsi list/Provinsi {country}.csv"
  new_data = pd.read_csv(path, sep=';')
  countries = pd.concat([countries, new_data], ignore_index=True)

countries.to_csv('/content/gdrive/MyDrive/WEB SCRAPING 101/List Country Full.csv', index=False)
'''

'\ncountries = pd.DataFrame()\ncountry_list = [\'Australia\', \'Indonesia\', \'Malaysia\', \'New Zealand\', \'Singapore\']\n\nfor country in country_list:\n  path = f"/content/gdrive/MyDrive/WEB SCRAPING 101/provinsi list/Provinsi {country}.csv"\n  new_data = pd.read_csv(path, sep=\';\')\n  countries = pd.concat([countries, new_data], ignore_index=True)\n\ncountries.to_csv(\'/content/gdrive/MyDrive/WEB SCRAPING 101/List Country Full.csv\', index=False)\n'

In [ ]:
path = '/content/gdrive/MyDrive/WEB SCRAPING 101/List Country Full.csv'
countries = pd.read_csv(path)
countries

,City,State,Country
0,Sydney,New South Wales,Australia
1,Melbourne,Victoria,Australia
2,Brisbane,Queensland,Australia
3,Perth,Western Australia,Australia
4,Adelaide,South Australia,Australia
...,...,...,...
216,Hougang,North East,Singapore
217,Woodlands,North,Singapore
218,Tampines,East,Singapore
219,Jurong West,West,Singapore


In [ ]:
list_country = ['Australia', 'Indonesia', 'Malaysia', 'New Zealand', 'Singapore']
def get_loc(x, dataframe=countries):
  new_loc = 'Unidentified'
  country = 'Unidentified'
  for i in list_country:
    substring = i
    fullstring = x
    if re.search(substring, fullstring):
      country = i
  #end
  if country != 'Unidentified':
    country_name =  country
    state_name = ''
    new_country_data = countries.loc[countries['Country'] == country].reset_index(drop=True)
    for i in range(len(new_country_data)):
      substring = new_country_data.iloc[i]['City']
      fullstring = x
      if re.search(substring, fullstring):
        state_name = new_country_data.iloc[i]['State']
      else:
        substring = new_country_data.iloc[i]['State']
        fullstring = x
        if re.search(substring, fullstring):
          state_name = new_country_data.iloc[i]['State']
      if state_name != '':
        break
    if state_name != '':
      return f"{state_name}, {country_name}"
    else:
      return f"{country_name}"
  else:
    return f"{country}"


In [ ]:
data['new_loc'] = data['location'].apply(get_loc, dataframe=countries)

In [ ]:
data.loc[data['new_loc']== 'Unidentified']

,job_title,company_name,location,date_posted,date_scraped,seniority,emp_type,industries,company_followers,company_size,total_applicants,work_place,job_link,job_desc,new_loc
7,US Expansion Intern (Remote),Campaign #ForABetterWorld,Jakarta Metropolitan Area,2023-08-02,2023-8-3,Internship,Internship,Civic and Social Organizations,14119,51-200 Employee,137,Remote,https://id.linkedin.com/jobs/view/us-expansion...,Campaign.com is expanding! We are looking for ...,Unidentified
12,Software Engineer Intern,Ninja Van,Jakarta Metropolitan Area,2023-07-24,2023-8-3,Internship,Internship,Truck Transportation,90181,10001+ Employee,2874,Hybrid,https://id.linkedin.com/jobs/view/software-eng...,Ninja Van is a late-stage logtech startup that...,Unidentified
14,Data Analyst,Stockbit,Jakarta Metropolitan Area,2023-07-27,2023-8-3,Entry level,Full-time,Capital Markets,26003,501-1000 Employee,240,Hybrid,https://id.linkedin.com/jobs/view/data-analyst...,nullResponsibilities\n\n\n\n\nServe as a strat...,Unidentified
15,Data Analyst,Astra Financial,Jakarta Metropolitan Area,2023-08-01,2023-8-3,Entry level,Full-time,Financial Services,46611,10001+ Employee,305,Hybrid,https://id.linkedin.com/jobs/view/data-analyst...,Menganalisis data dan informasi bisnis dari be...,Unidentified
17,Data Analyst,Traveloka,Jakarta Metropolitan Area,2023-07-11,2023-8-3,Entry level,Full-time,Software Development,538901,1001-5000 Employee,1607,Onsite,https://id.linkedin.com/jobs/view/data-analyst...,It's fun to work in a company where people tru...,Unidentified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Senior Web Developer,Matomo,Greater Tauranga Area,2023-07-21,2023-8-8,Not Applicable,Full-time,"Transportation, Logistics, Supply Chain and St...",5783,11-50 Employee,38,Remote,https://nz.linkedin.com/jobs/view/senior-web-d...,Job Description\n\n\n\n\nWe are seeking a tale...,Unidentified
18080,Senior AWS Cloud Engineer - Enterprise Migrati...,84 Recruitment,Greater Kapiti Area,2023-07-25,2023-8-8,Mid-Senior level,Contract,IT Services and IT Consulting,6588,11-50 Employee,13,Remote,https://nz.linkedin.com/jobs/view/senior-aws-c...,About The Company\n\n\n\n\nOur client is a one...,Unidentified
18086,Senior AWS Cloud Engineer - Enterprise Migrati...,84 Recruitment,Christchurch Metropolitan Area,2023-07-25,2023-8-8,Mid-Senior level,Contract,IT Services and IT Consulting,6588,11-50 Employee,12,Remote,https://nz.linkedin.com/jobs/view/senior-aws-c...,About The Company\n\n\n\n\nOur client is a one...,Unidentified
18155,Full Stack PHP/JS Engineer Team Lead,Matomo,Greater Hamilton Area,2023-03-10,2023-8-8,Not Applicable,Full-time,"Transportation, Logistics, Supply Chain and St...",5771,11-50 Employee,159,Remote,https://nz.linkedin.com/jobs/view/full-stack-p...,We’re on the lookout for a Developer Team Lead...,Unidentified


In [ ]:
dict_loc = {
    'Jakarta Metropolitan Area': 'DKI Jakarta, Indonesia',
    'Greater Kuala Lumpur': 'Wilayah Persekutuan Kuala Lumpur, Malaysia',
    'Greater Sydney Area': 'New South Wales, Australia',
    'Greater Melbourne Area': 'Victoria, Australia',
    'Greater Perth Area': 'Western Australia, Australia',
    'Greater Johor Bahru': 'Johor, Malaysia',
    'Greater Ipoh': 'Perak, Malaysia',
    'Greater Brisbane Area': 'Queensland, Australia',
    'Greater Yogyakarta': 'Daerah Istimewa Yogyakarta, Indonesia',
    'Greater Newcastle Area': 'New South Wales, Australia',
    'Greater Kapiti Area': 'New Zealand',
    'Christchurch Metropolitan Area': 'Canterbury, New Zealand',
    'Greater Tauranga Area': 'Bay of Plenty, New Zealand',
    'Greater Hamilton Area': 'Waikato, New Zealand',
    'Greater Alor Setar': 'Kedah, Malaysia',
    'Rockhampton Area': 'Queensland, Australia',
    'Wollongong Area': 'New South Wales, Australia'
}

In [ ]:
for key,value in dict_loc.items():
  data.loc[(data['location'] == key), 'new_loc'] = value

In [ ]:
data.loc[data['new_loc']== 'Unidentified']

,job_title,company_name,location,date_posted,date_scraped,seniority,emp_type,industries,company_followers,company_size,total_applicants,work_place,job_link,job_desc,new_loc


In [ ]:
def get_state(x):
  substring = ','
  fullstring = x
  if re.search(substring, fullstring):
    state = x.split(',')[0]
    return state
  else:
    return 'Unidentified'

def get_country(x):
  substring = ','
  fullstring = x
  if re.search(substring, fullstring):
    country = x.split(',')[1]
    country = country.replace(country[0], "", 1)
    return country
  else:
    return x

In [ ]:
data['state'] = data['new_loc'].apply(get_state)
data['country'] = data['new_loc'].apply(get_country)

In [ ]:
data.country.value_counts(normalize=True)

Singapore      0.275969
Australia      0.260623
Malaysia       0.215074
Indonesia      0.180229
New Zealand    0.068105
Name: country, dtype: float64

## 1.5 Cleaning `seniority` column

In [ ]:
data.seniority.value_counts()

Mid-Senior level                              7299
Entry level                                   6205
Associate                                     1857
\n            Entry level\n                    927
\n            Mid-Senior level\n               593
Internship                                     447
Full-time                                      368
Not Applicable                                 147
Director                                       143
\n            Associate\n                      121
Contract                                        76
\n            Internship\n                      52
\n          Full-time\n                         29
\n            Director\n                        18
Part-time                                       11
\n          Contract\n                          10
Temporary                                        2
Executive                                        2
\n          Part-time\n                          1
\n          Internship\n       

In [ ]:
list_sen = ['Mid-Senior level', 'Entry level', 'Associate', 'Internship', 'Full-time', 'Not Applicable', 'Director', 'Contract', 'Part-time', 'Executive', 'Temporary']

def clean_sen(x, lists=list_sen):
  for sen in list_sen:
    substring = sen
    fullstring = x
    if re.search(substring, fullstring):
      return sen

In [ ]:
data['new_seniority'] = data['seniority'].apply(clean_sen, lists=list_sen)

In [ ]:
data.new_seniority.value_counts()

Mid-Senior level    7892
Entry level         7132
Associate           1978
Internship           500
Full-time            397
Director             161
Not Applicable       147
Contract              86
Part-time             12
Temporary              3
Executive              2
Name: new_seniority, dtype: int64

In [ ]:
dict_sen={
  'Full-time': 'Entry level',
  'Not Applicable': 'Entry level',
  'Contract': 'Internship',
  'Part-time': 'Internship',
  'Temporary': 'Internship',
  'Executive': 'Director',
  'Associate': 'Entry level'
}

In [ ]:
for key,value in dict_sen.items():
  data.loc[(data['new_seniority'] == key), 'new_seniority'] = value

In [ ]:
data.new_seniority.value_counts()

Entry level         9654
Mid-Senior level    7892
Internship           601
Director             163
Name: new_seniority, dtype: int64

## 1.6 Transform `date_posted` into datetime

In [ ]:
data['date_posted'] = pd.to_datetime(data['date_posted'])

## 1.7 Cleaning `emp_type`

In [ ]:
data.emp_type.value_counts()

Full-time                           14571
\n          Full-time\n              1551
Contract                             1077
Unidentified                          505
Internship                            312
\n          Contract\n                 95
Part-time                              75
Temporary                              48
\n          Internship\n               37
Other                                  17
\n          Part-time\n                12
\n          Temporary\n                 8
\n          Other\n                     2
Name: emp_type, dtype: int64

In [ ]:
list_emp = ['Internship', 'Full-time', 'Not Applicable', 'Contract', 'Unidentified', 'Part-time', 'Temporary', 'Other']

def clean_emp(x, lists=list_emp):
  for emp in list_emp:
    substring = emp
    fullstring = x
    if re.search(substring, fullstring):
      return emp

In [ ]:
data['new_emp_type'] = data['emp_type'].apply(clean_emp, lists=list_emp)

In [ ]:
data.loc[data['new_emp_type'] == 'Temporary', 'new_emp_type'] = 'Contract'

In [ ]:
data.new_emp_type.value_counts()

Full-time       16122
Contract         1228
Unidentified      505
Internship        349
Part-time          87
Other              19
Name: new_emp_type, dtype: int64

# 1.8 Cleaning other columns

In [ ]:
def clean_text(x):
  value = re.sub('\n', ' ', x)
  value = re.sub(' +', ' ', value)
  value = value.strip()
  return value

In [ ]:
data['job_description'] = data['job_desc'].apply(clean_text)
data['job_title'] = data['job_title'].apply(clean_text)
data['company_name'] = data['company_name'].apply(clean_text)
data['industries'] = data['industries'].apply(clean_text)

In [ ]:
data.columns

Index(['job_title', 'company_name', 'location', 'date_posted', 'date_scraped',
       'seniority', 'emp_type', 'industries', 'company_followers',
       'company_size', 'total_applicants', 'work_place', 'job_link',
       'job_desc', 'new_loc', 'state', 'country', 'new_seniority',
       'new_emp_type', 'job_description'],
      dtype='object')

In [ ]:
data.head(2)

,job_title,company_name,location,date_posted,date_scraped,seniority,emp_type,industries,company_followers,company_size,total_applicants,work_place,job_link,job_desc,new_loc,state,country,new_seniority,new_emp_type,job_description
0,Data Science Trainee,Algoritma Data Science School,"Jakarta, Jakarta, Indonesia",2023-07-01,2023-8-3,Internship,Full-time,Education Administration Programs,11560,11-50 Employee,155,Onsite,https://id.linkedin.com/jobs/view/data-science...,Background\n\n\n\n\nAre you looking to build a...,"DKI Jakarta, Indonesia",DKI Jakarta,Indonesia,Internship,Full-time,Background Are you looking to build a career i...
1,Data Intelligence Intern,SIRCLO,"Tangerang, Banten, Indonesia",2023-08-02,2023-8-3,Internship,Full-time,IT Services and IT Consulting,67415,1001-5000 Employee,126,Hybrid,https://id.linkedin.com/jobs/view/data-intelli...,nullResponsibilities\n\n\n\n\nSIRCLO Internshi...,"Banten, Indonesia",Banten,Indonesia,Internship,Full-time,nullResponsibilities SIRCLO Internship Program...


In [ ]:
data.drop(['location', 'seniority', 'emp_type', 'job_desc', 'new_loc'], axis=1, inplace=True)

In [ ]:
data.rename(columns={
    'new_seniority' : 'seniority_level',
    'new_emp_type' : 'employment_type'
    },
    inplace = True)

In [ ]:
data = data[['job_title', 'company_name', 'date_posted', 'date_scraped', 'seniority_level', 'employment_type', 'work_place', 'industries', 'state', 'country', 'company_size', 'company_followers', 'total_applicants', 'job_link', 'job_description']]

In [ ]:
data.sample()

,job_title,company_name,date_posted,date_scraped,seniority_level,employment_type,work_place,industries,state,country,company_size,company_followers,total_applicants,job_link,job_description
14,Data Analyst,Stockbit,2023-07-27,2023-8-3,Entry level,Full-time,Hybrid,Capital Markets,DKI Jakarta,Indonesia,501-1000 Employee,26003,240,https://id.linkedin.com/jobs/view/data-analyst...,nullResponsibilities Serve as a strategic advi...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18310 entries, 0 to 18309
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   job_title          18310 non-null  object        
 1   company_name       18310 non-null  object        
 2   date_posted        18310 non-null  datetime64[ns]
 3   date_scraped       18310 non-null  object        
 4   seniority_level    18310 non-null  object        
 5   employment_type    18310 non-null  object        
 6   work_place         18310 non-null  object        
 7   industries         18310 non-null  object        
 8   state              18310 non-null  object        
 9   country            18310 non-null  object        
 10  company_size       18310 non-null  object        
 11  company_followers  18310 non-null  int64         
 12  total_applicants   18310 non-null  int64         
 13  job_link           18310 non-null  object        
 14  job_de

In [ ]:
'''data.to_csv('/content/gdrive/MyDrive/WEB SCRAPING 101/Data Cleaned_new.csv', index=False)'''